# 1) Web Scraping

In [65]:
#main libraries
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
#import folium
import requests
from geopy.geocoders import Nominatim

In [2]:
#prepare webscraper 
!pip install beautifulsoup4
!pip install lxml
from bs4 import BeautifulSoup
import csv
import json
import lxml

In [3]:
#get raw html file from url page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url).text
mySoup = BeautifulSoup(results, 'lxml')

In [4]:
scrapTable = mySoup.body.find('table')

In [5]:
table_rows = scrapTable.findAll('tr')

In [6]:
output_rows = []
for table_row in table_rows:
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        if(column.text != 'Not assigned'):
            output_row.append(column.text)
        else:
            output_row.append('discard')
    output_rows.append(output_row)


In [7]:
#initialize .csv file on which we will write the data from the webpage and that we will later transform in a dataframe
csv_file = open('TorontoData.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerows(output_rows)

In [8]:
csv_file.close()

# 2) Create dataframe

In [9]:
df = pd.read_csv('TorontoData.csv')

In [10]:
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

In [11]:
df = df[df.Borough != 'discard']

In [12]:
df.head(20)

,PostalCode,Borough,Neighborhood
1,M3A,North York,Parkwoods\n
2,M4A,North York,Victoria Village\n
3,M5A,Downtown Toronto,Harbourfront\n
4,M5A,Downtown Toronto,Regent Park\n
5,M6A,North York,Lawrence Heights\n
6,M6A,North York,Lawrence Manor\n
7,M7A,Queen's Park,Not assigned\n
9,M9A,Etobicoke,Islington Avenue\n
10,M1B,Scarborough,Rouge\n
11,M1B,Scarborough,Malvern\n


In [13]:
TorontoDF = pd.DataFrame(columns= ['PostalCode', 'Borough', 'Neighborhood'])

In [14]:
for index, row in df.iterrows():
    if(row['Neighborhood'] == 'Not assigned\n'):
        row['Neighborhood'] = row['Borough']
    postCode = row['PostalCode']
    borough = row['Borough']
    neighborhood = row['Neighborhood']
    
    TorontoDF = TorontoDF.append({'PostalCode': postCode, 'Borough': borough, 'Neighborhood': neighborhood}, ignore_index=True)

In [15]:
TorontoDF['Neighborhood'] = TorontoDF['Neighborhood'].str.replace('\n','')

In [32]:
TorontoDF.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [61]:
#TorontoDF.groupby(['PostalCode']).groups.keys()
#TorAggregated = TorontoDF.groupby(['PostalCode'])[['Neighborhood'], ['Borough']].apply(lambda x: ','.join(x)).reset_index()
TorAggregated = TorontoDF.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list)

In [62]:
TorontoDF_Final= pd.DataFrame(TorAggregated)

In [63]:
TorontoDF_Final.reset_index(inplace=True)

In [64]:
TorontoDF_Final.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


In [79]:
TorontoDF_Final.shape

(103, 3)